In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [3]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
file_name="tangail_words"
df = pd.read_excel(f"{file_name}.xlsx")
df['ipa'] = None
print(df.head())
print(df.shape)

  words   ipa
0  যদিও  None
1   এতো  None
2    বড়  None
3  আগুন  None
4  লাগে  None
(9494, 2)


In [14]:
df.head()

,words,ipa
0,অনর,ɔnɔr
1,বুঝ,buɟʱ
2,ফঅলেও,pʰɔ͡o̯le͡o̯
3,বুঝে,buɟʱe
4,অইনত্তোবারে,ɔ͡i̯not̪t̪obare


In [4]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    # print(f"Started {row.Index}")
    if pd.notna(row.ipa) and "ERROR" not in row.ipa:
      print("exist")
    else:
      print(f"Started {row.Index} {row.words}")
      # words=row.sentence.split(" ")
      if(False):
        datas=""
        for word in words:
          # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
          url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
          response=None
          try:
            response = requests.get (url, verify=False)
            if response and response.status_code == 200:
              data= response.json()
              datas=datas+" "+data["data"][0]
            else:
              datas=datas+" "+"ERROR"
          except:
            print(f"error at{row.Index}:{row.sentence}")
            datas=datas+" "+"ERROR"
        print(f"Ended {row.Index}")
        df.at[row.Index,"ipa"]=datas
      else:
        url = f"http://127.0.0.1:8001//post_text_bangla_ipa?bangla_ipa={row.words}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            df.at[row.Index,"ipa"]=data["data"][0]
            print(f"Ended {row.Index}:{row.words}: {data['data'][0]}")
          else:
            data= None
        except Exception as e:
          print(e)
          print(f"error at{row.Index}:{row.words}")
    
    # if pd.notna(row.ipa_rangpur) and "ERROR" not in row.ipa_rangpur:
    #   continue
    # print(f"Started {row.Index} {row.Rangpur}")
    # if(False):
    #   datas=""
    #   for word in words:
    #     # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
    #     url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
    #     response=None
    #     try:
    #       response = requests.get (url, verify=False)
    #       if response and response.status_code == 200:
    #         data= response.json()
    #         datas=datas+" "+data["data"][0]
    #       else:
    #         datas=datas+" "+"ERROR"
    #     except:
    #       print(f"error at{row.Index}:{row.sentence}")
    #       datas=datas+" "+"ERROR"
    #   print(f"Ended {row.Index}")
    #   df.at[row.Index,"ipa"]=datas
    # else:
    #   url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.Rangpur}"
    #   response=None
    #   try:
    #     response = requests.get (url, verify=False)
    #     if response and response.status_code == 200:
    #       data= response.json()
    #       df.at[row.Index,"ipa_rangpur"]=data["data"][0]
    #       print(f"Ended {row.Index}:{row.Rangpur}")
    #       print(f"{row.Index}:{data['data'][0]}")
    #     else:
    #       data= None
    #   except Exception as e:
    #     print(e)
    #     print(f"error at{row.Index}:{row.Rangpur}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [5]:
def subset_worker(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started{row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.sentence}"
    response=None
    try:
      response = requests.get (url, verify=False)
      if response and response.status_code == 200:
        data= response.json()
        df.at[row.Index,"ipa"]=data["data"][0]
      else:
        data= None
    except:
      print(f"error at{row.Index}:{row.sentence}")
    print(f"Ended {row.Index}")
    # print(f"{row.Index}:{row.ipa}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [6]:
def subset_worker_db(subset):
  for row in subset.itertuples():
    data=call_api(row.word)
    if data:
      print(f"{row.Index}:{data['data'][0]}")
    else:
      print(row.Index)
      print(row.word)
      break

In [5]:
# threadpool make sure everything is currect before running this cell

i=0;
while(i<=10000):
  start=i
  steps=200
  end=i+2000
  subsets=[]
  start_pos=start
  while start<end:
    subsets.append(df.iloc[start:start+steps])
    start=start+steps
  print(subsets)
  pool=ThreadPoolExecutor(len(subsets))
  ws=[]
  for subset in subsets:
    ws.append(pool.submit(subset_worker_words, subset))
  print(pool.shutdown())
  df.to_csv(f"{file_name}_{start_pos}_{end}.csv",index=False)
  i=i+2000

[    words   ipa
0    যদিও  None
1     এতো  None
2      বড়  None
3    আগুন  None
4    লাগে  None
..    ...   ...
195  বলোস  None
196   আমি  None
197  তোরে  None
198   বলি  None
199   গরম  None

[200 rows x 2 columns],              words   ipa
200        বাহিরের  None
201           দেশে  None
202  সুইজারল্যান্ড  None
203             বল  None
204           ইউকে  None
..             ...   ...
395            দিন  None
396          হিসাব  None
397         মানতাম  None
398           সাড়ে  None
399      লাইনম্যান  None

[200 rows x 2 columns],            words   ipa
400  সুপারভাইজার  None
401       এইগুলা  None
402          হেড  None
403            ও  None
404       অইতাছে  None
..           ...   ...
595         পারি  None
596      হোটেলের  None
597        নামটা  None
598        আঠারো  None
599      ক্যারেট  None

[200 rows x 2 columns],          words   ipa
600   স্বর্ণের  None
601     বিক্রি  None
602       কেজি  None
603       যেনো  None
604        বিশ  None
..         ...   ...
795      

In [ ]:

start=0
steps=200
end=2000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
print(subsets)
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_words, subset))
print(pool.shutdown())
df.to_csv(f"{file_name}_{start_pos}_{end}.csv",index=False)

In [ ]:
df.to_excel(f"{file_name}_withIPA.xlsx",index=False)

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[3000:6000]
for row in subset1_1000.itertuples():
  print(row.Index)
  # print(row[1])
  if pd.notna(row.ipa):
    continue
  print(row.sentence)
  data=call_api(row.sentence)
  # print(data["data"][0])
  if data:
    df.at[row.Index,"ipa"]=data["data"][0]
  else:
    print(row.Index)
    print(row.sentence)
    break
df.to_csv("withIPA.csv",index=False)

**TESTINS**

---



In [3]:
# test connection
def call_api (param):
    url = f"http://127.0.0.1:5000//transcribeBanglaTextToIpa?bangla_ipa={param}"
    response = requests.get (url, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        return None
data=call_api("আমার সোনার বাংলা।")
data["data"][0]

'amar ʃonar baŋla।'

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[20000:70000]
for row in subset1_1000.itertuples():
  # print(row.Index)
  # print(row[1])
  # if pd.notna(row.ipa):
  #   continue
  # print(row.word)
  data=call_api(row.word)
  # print(data["data"][0])
  if data:
    # df.at[row.Index,"ipa"]=data["data"][0]
    print(f"{row.Index}:{data['data'][0]}")
  else:
    print(row.Index)
    print(row.word)
    break
# df.to_csv("withIPA.csv",index=False)

In [ ]:
multiprocessing.cpu_count()

In [43]:
print(pool.shutdown( cancel_futures=True))
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

None


In [ ]:

# w0= pool.submit(subset_worker, subsets[0])
# w1=pool.submit(subset_worker, subsets[1])
# w2=pool.submit(subset_worker, subsets[2])
# w3=pool.submit(subset_worker, subsets[3])
# w4=pool.submit(subset_worker, subsets[4])
# w0.result()
# w1.result()
# w2.result()
# w3.result()
# w4.result()

# print(w0.cancel())
# print(w1.cancel())
# print(w2.cancel())
# print(w3.cancel())
# print(w4.cancel())


In [ ]:
# not working
i=500
steps=100
end=1000
subsets=[]
while i<end:
  subsets.append(df.iloc[i:i+steps])
  i=i+steps
print(subsets)

threads=[]
for subset in subsets:
  threads.append(threading.Thread(target=subset_worker, daemon=True, args=(subset,)))
for thread in threads:
  thread.start();
  time.sleep(1)
for thread in threads:
  thread.join()
print("end")

Cleaning

In [97]:
# Unique word clearing
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
df.drop_duplicates(subset=['bangla_token'], inplace=True)
print(len(df))
df.sort_values('bangla_token').to_csv("bangla_to_ipa_unique_clean.csv",index=False)



58569


In [99]:
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if len(row.bangla_token)>2 and row.bangla_token[-2]=="!":
    # print(row.bangla_token[1:])

    for i in range(1,110,1):
      if df.at[row.Index-i,"bangla_token"]==row.bangla_token[:-2]:
        print(row.Index)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break

df.drop(remove, inplace=True)
print(len(remove))

0


In [113]:
# check whole dataset
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if row.bangla_token[:-1]=="?":
    # if contains in the whole dataset
    for i in range(len(df)):
      if df.at[i,"bangla_token"]==row.bangla_token[:-1]:
        print(row.Index)
        print(i)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break
df.drop(remove, inplace=True)


In [110]:
df.to_csv("bangla_to_ipa_unique_clean2.csv",index=False)

In [83]:
a="bangla_to_ipa_unique_clean"
print(a[1:])

angla_to_ipa_unique_clean
